## General requests
- comment the code
- code able to run on Linux/macOS/unix-like environment
- way to save and load the entire index from disk (avoid re-indexing when the program starts)
- consider performance

## Tasks
Write an IR system using the Vector Space Model:
1. able to answer free-form text queries 
2. allowing relevance feedback
3. allowing the use of pseudo-relevance feedback
4. evaluate it on a set of test queries

## Model
Each document is seen as a vector with:
- a component for each term in the dictionary
- as elements the tf-idf $_{t,d}$ of the term $t$ in the document

tf-idf $_{t,d}$ = 0 for all terms not in the document

### To score a document
Sum the tf-idf $_{t,d}$ values for all terms appeaing in $q$
$$ Score(q,d) = \sum_{t \in q} tf-idf_{t,d} $$
### Terms
Elements of the canonical base of $ \mathcal R ^n $ -> n = number of terms in the dictionary

### Documents
n-dimensional vector with each element the $tf-idf_{t,d}$

Document vectors can be normalized and be replaced with a unit vector -> $ v(d) = \frac{V(d)}{|V(d)|} $

### Queries
Unit vector with the non-zero components correspoding to the query terms

### Cosine similarity 
1. to compare documents: $ sim(d_1, d_2) = \frac{V(d_1) V(d_2)}{|V(d_1)| |V(d_2)|} $ -> the cosine of the angle formed by the two vectors

2. to answer queries: $ score(q,d_1)=v(q) v(d_1) \equiv score(q,d_1)=v(q) v(d_1) $

In [1]:
from collections import defaultdict, OrderedDict
from math import log, sqrt
from os import listdir
from os.path import isfile, join
import re
import glob

### Implement the function to import the dataset!

In [2]:
def import_dataset():
    """
    This function import all the articles in the TIME corpus,
    returning list of lists where each sub-list contains all the
    terms present in the document as a string.
    """
    articles = []
    with open('TIME.ALL', 'r') as f:
        tmp = []
        for row in f:
            if row.startswith("*TEXT"):
                if tmp != []:
                    articles.append(tmp)
                tmp = []
            else:
                row = re.sub(r'[^a-zA-Z\s]+', '', row)
                tmp += row.split()
    return articles

In [4]:
corpus = import_dataset()
corpus_small = corpus[:5]
corpus_small

[['THE',
  'ALLIES',
  'AFTER',
  'NASSAU',
  'IN',
  'DECEMBER',
  'THE',
  'US',
  'FIRST',
  'PROPOSED',
  'TO',
  'HELP',
  'NATO',
  'DEVELOP',
  'ITS',
  'OWN',
  'NUCLEAR',
  'STRIKE',
  'FORCE',
  'BUT',
  'EUROPE',
  'MADE',
  'NO',
  'ATTEMPT',
  'TO',
  'DEVISE',
  'A',
  'PLAN',
  'LAST',
  'WEEK',
  'AS',
  'THEY',
  'STUDIED',
  'THE',
  'NASSAU',
  'ACCORD',
  'BETWEEN',
  'PRESIDENT',
  'KENNEDY',
  'AND',
  'PRIME',
  'MINISTER',
  'MACMILLAN',
  'EUROPEANS',
  'SAW',
  'EMERGING',
  'THE',
  'FIRST',
  'OUTLINES',
  'OF',
  'THE',
  'NUCLEAR',
  'NATO',
  'THAT',
  'THE',
  'US',
  'WANTS',
  'AND',
  'WILL',
  'SUPPORT',
  'IT',
  'ALL',
  'SPRANG',
  'FROM',
  'THE',
  'ANGLOUS',
  'CRISIS',
  'OVER',
  'CANCELLATION',
  'OF',
  'THE',
  'BUGRIDDEN',
  'SKYBOLT',
  'MISSILE',
  'AND',
  'THE',
  'US',
  'OFFER',
  'TO',
  'SUPPLY',
  'BRITAIN',
  'AND',
  'FRANCE',
  'WITH',
  'THE',
  'PROVED',
  'POLARIS',
  'TIME',
  'DEC',
  'THE',
  'ONE',
  'ALLIED',
  'LEADER

Function to compute the positional index: returns a dictionary in which for each term we have a dictionary with as keys the docIDs (integers referring to one document) and as values a list with the position of the term in the document.

$ \texttt{ \{"RUSSIA" : \{0:[1,6,35], 10:[6,22,105]\}, "COLD" : \{12:[6], 100:[2]\}\} } $

In [5]:
def pos_index(corpus):
    p_idx = defaultdict(dict)
    for docid, d in enumerate(corpus):  # for each document in the corpus
        for pos, t in enumerate(d):  # for each term in the document
            t_idx = p_idx.get(t, False)
            if not t_idx:
                p_idx[t][docid] = [pos]
            else:
                d_idx = p_idx[t].get(docid, False)
                if not d_idx:
                    p_idx[t][docid] = [pos]
                else:
                    p_idx[t][docid].append(pos)
    return p_idx

In [6]:
p_idx = pos_index(corpus_small)
p_idx

defaultdict(dict,
            {'THE': {0: [0,
               6,
               33,
               46,
               50,
               54,
               64,
               70,
               75,
               84,
               89,
               96,
               120,
               135,
               148,
               154,
               183,
               219,
               223,
               285,
               295,
               315,
               317,
               341,
               348,
               365,
               378,
               402,
               432,
               448,
               470,
               486,
               495,
               514,
               531,
               595,
               606,
               655,
               664,
               692,
               727,
               739,
               743,
               750,
               762,
               775,
               822,
               827,
               831,
      

Function to compute the inverse document frequency for each term. Returns a dictionary with terms as keys and IDF as values.
$ \texttt{ \{"THE":0.0, "REVENUES": 1.6094379124341003\}} $

In [7]:
def inverse_doc_freq(p_idx, n_docs):
    idf = dict()
    for t in p_idx.keys():
        idf[t] = log( n_docs / len(p_idx[t]) )
    return idf

In [8]:
n_docs = len(corpus_small)
idf = inverse_doc_freq(p_idx, n_docs)
idf

{'THE': 0.0,
 'ALLIES': 1.6094379124341003,
 'AFTER': 0.5108256237659907,
 'NASSAU': 1.6094379124341003,
 'IN': 0.0,
 'DECEMBER': 1.6094379124341003,
 'US': 0.22314355131420976,
 'FIRST': 0.22314355131420976,
 'PROPOSED': 1.6094379124341003,
 'TO': 0.0,
 'HELP': 1.6094379124341003,
 'NATO': 1.6094379124341003,
 'DEVELOP': 1.6094379124341003,
 'ITS': 0.5108256237659907,
 'OWN': 0.9162907318741551,
 'NUCLEAR': 1.6094379124341003,
 'STRIKE': 0.9162907318741551,
 'FORCE': 0.9162907318741551,
 'BUT': 0.5108256237659907,
 'EUROPE': 1.6094379124341003,
 'MADE': 1.6094379124341003,
 'NO': 0.5108256237659907,
 'ATTEMPT': 0.9162907318741551,
 'DEVISE': 1.6094379124341003,
 'A': 0.0,
 'PLAN': 0.9162907318741551,
 'LAST': 0.22314355131420976,
 'WEEK': 0.9162907318741551,
 'AS': 0.22314355131420976,
 'THEY': 0.22314355131420976,
 'STUDIED': 1.6094379124341003,
 'ACCORD': 1.6094379124341003,
 'BETWEEN': 0.9162907318741551,
 'PRESIDENT': 1.6094379124341003,
 'KENNEDY': 0.9162907318741551,
 'AND': 0.0

Function to compute the TF-IDF for each term and each document. Returns a dictionary with docIDs as keys and as values a dictionary with terms as keys and as values the corresponding tf-idf for the specific term in the specific document.

In this way, each document is seen as a n-dimensional vector, with n being the number of terms in the dictionary.

$ \texttt{ \{0:\{"THE": 0.0\}, 10:\{"THE": 1.6094379124341003\}\} } $

In [9]:
def tf_idf(docs, p_idx):
    # p_idx = pos_index(docs)
    tfidf = {}
    n_docs = len(docs)
    idf = inverse_doc_freq(p_idx, n_docs)

    for d in range(len(docs)):  #for each document
        doc_vector = {}  # we store for each term in the document the tfidf 
                         # and for those not in the document a 0
        for t in p_idx.keys():
            if t in docs[d]:
                doc_vector[t] = idf[t] * len(p_idx[t][d]) 
            else:
                doc_vector[t] = 0
        tfidf[d] = doc_vector
    return tfidf

In [10]:
tfidf = tf_idf(corpus_small, p_idx)
tfidf

{0: {'THE': 0.0,
  'ALLIES': 3.2188758248682006,
  'AFTER': 1.5324768712979722,
  'NASSAU': 6.437751649736401,
  'IN': 0.0,
  'DECEMBER': 1.6094379124341003,
  'US': 3.3471532697131465,
  'FIRST': 0.8925742052568391,
  'PROPOSED': 1.6094379124341003,
  'TO': 0.0,
  'HELP': 4.828313737302301,
  'NATO': 9.656627474604601,
  'DEVELOP': 3.2188758248682006,
  'ITS': 3.5757793663619353,
  'OWN': 3.6651629274966204,
  'NUCLEAR': 16.094379124341003,
  'STRIKE': 1.8325814637483102,
  'FORCE': 8.246616586867397,
  'BUT': 1.5324768712979722,
  'EUROPE': 4.828313737302301,
  'MADE': 1.6094379124341003,
  'NO': 2.5541281188299534,
  'ATTEMPT': 0.9162907318741551,
  'DEVISE': 1.6094379124341003,
  'A': 0.0,
  'PLAN': 0.9162907318741551,
  'LAST': 0.8925742052568391,
  'WEEK': 2.7488721956224653,
  'AS': 0.8925742052568391,
  'THEY': 0.6694306539426294,
  'STUDIED': 1.6094379124341003,
  'ACCORD': 1.6094379124341003,
  'BETWEEN': 0.9162907318741551,
  'PRESIDENT': 3.2188758248682006,
  'KENNEDY': 3.6

Function to convert a query, given as a free-form text, to a n-dimensional vector, with n being the number of terms in the dictionary, with 1 if the term is present in the query and 0 otherwise.

$ \texttt{query = "revenues tree"} $

$ \texttt{ \{"THE":0, "REVENUES":1\} } $


In [11]:
def query_as_vector(q, p_idx):
    q = q.upper().split(" ")
    q_vec = {}
    for t in p_idx:
        if t in q:
            q_vec[t] = 1
        else:
            q_vec[t] = 0
    return q_vec


In [14]:
query = "revenues tree"
q_vec = query_as_vector(query, p_idx)
q_vec

{'THE': 0,
 'ALLIES': 0,
 'AFTER': 0,
 'NASSAU': 0,
 'IN': 0,
 'DECEMBER': 0,
 'US': 0,
 'FIRST': 0,
 'PROPOSED': 0,
 'TO': 0,
 'HELP': 0,
 'NATO': 0,
 'DEVELOP': 0,
 'ITS': 0,
 'OWN': 0,
 'NUCLEAR': 0,
 'STRIKE': 0,
 'FORCE': 0,
 'BUT': 0,
 'EUROPE': 0,
 'MADE': 0,
 'NO': 0,
 'ATTEMPT': 0,
 'DEVISE': 0,
 'A': 0,
 'PLAN': 0,
 'LAST': 0,
 'WEEK': 0,
 'AS': 0,
 'THEY': 0,
 'STUDIED': 0,
 'ACCORD': 0,
 'BETWEEN': 0,
 'PRESIDENT': 0,
 'KENNEDY': 0,
 'AND': 0,
 'PRIME': 0,
 'MINISTER': 0,
 'MACMILLAN': 0,
 'EUROPEANS': 0,
 'SAW': 0,
 'EMERGING': 0,
 'OUTLINES': 0,
 'OF': 0,
 'THAT': 0,
 'WANTS': 0,
 'WILL': 0,
 'SUPPORT': 0,
 'IT': 0,
 'ALL': 0,
 'SPRANG': 0,
 'FROM': 0,
 'ANGLOUS': 0,
 'CRISIS': 0,
 'OVER': 0,
 'CANCELLATION': 0,
 'BUGRIDDEN': 0,
 'SKYBOLT': 0,
 'MISSILE': 0,
 'OFFER': 0,
 'SUPPLY': 0,
 'BRITAIN': 0,
 'FRANCE': 0,
 'WITH': 0,
 'PROVED': 0,
 'POLARIS': 0,
 'TIME': 0,
 'DEC': 0,
 'ONE': 0,
 'ALLIED': 0,
 'LEADER': 0,
 'WHO': 0,
 'UNRESERVEDLY': 0,
 'WELCOMED': 0,
 'WAS': 0,


Function to compute the relevance score for each document given a query. The score is computed as the cosine similarity between the query vector and the document vector. It returns a dictionary with the docIDs as keys and the relevance score as values.

$ \texttt{ \{0:0.0, 1:0.03272921367166952\} } $

In [15]:
def relevance_scores(query, p_idx, docs):
    scores = dict()  # for each document we store the cosine similarity
                     # between the query and the document
    q = query_as_vector(query, p_idx)
    tfidf = tf_idf(docs, p_idx)
    for d in range(len(docs)):
        cos_sim = 0
        for t in q:
            cos_sim += tfidf[d][t] * q[t]
        scores[d] = cos_sim / (sqrt(sum([x**2 for x in q.values()])) * \
            sqrt(sum([x**2 for x in tfidf[d].values()])))
    
    return scores


In [16]:
scores = relevance_scores(query, p_idx, corpus_small)
scores

{0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 0.03272921367166952}

Function to return the k most relevant documents given a query.



In [17]:
def vector_space_model(query, k):
    # for a given query and the tfidf matrix, 
    # we return the top k documents
    # relevant for the given query

    # corpus = import_dataset()
    corpus = corpus_small

    p_idx = pos_index(corpus)

    scores = relevance_scores(query, p_idx, corpus)
    sorted_value = OrderedDict(sorted(scores.items(), key=lambda x: x[1], reverse=True))
    topk = {key : sorted_value[key] for key in list(sorted_value)[:k] if sorted_value[key]!=0}

    # if len(topk) == k:
    #     print(f"The {k} documents with highest score are {topk}")
    # if len(topk) < k:
    #     print(f"Some documents reported a score equal to 0.\nThe documents with highest score are {[(key, value) for key, value in topk.items()]}")

    return topk

In [18]:
topk_docs = vector_space_model(query, 3)
topk_docs

{4: 0.03272921367166952}

# Relevance and pseudo-relevance feedback

Feedback given as a query and a list of `(docID, feedback_value)` with 0. means irrelevant and 1. means relevant.

In [ ]:
feedback = {
        'who makes chatbots': [(2, 0.), (0, 1.), (1, 1.), (0, 1.)],
        'about page': [(0, 1.)]
}

In [22]:
vocab = list(p_idx.keys())

In [25]:
def relevance_feedback_rocchio(rel_docs, nrel_docs, query, tfidf, vocab, alpha=1, beta=0.75, gamma=0.15):
    q_opt = dict()
    for t in vocab:
        r = 0
        for docid in rel_docs:
            r += tfidf[docid]
        r /= len(rel_docs)

        if len(nrel_docs) != 0:
            n = 0
            for docid in nrel_docs:
                n += tfidf[docid]
            n /= len(nrel_docs)
        else:
            gamma = 0

        opt = alpha*query[t] + beta*r - gamma*n
        if opt < 0:
            q_opt[t] = 0
        else:
            q_opt[t] = opt

    return q_opt

In [24]:
q_vec

{'THE': 0,
 'ALLIES': 0,
 'AFTER': 0,
 'NASSAU': 0,
 'IN': 0,
 'DECEMBER': 0,
 'US': 0,
 'FIRST': 0,
 'PROPOSED': 0,
 'TO': 0,
 'HELP': 0,
 'NATO': 0,
 'DEVELOP': 0,
 'ITS': 0,
 'OWN': 0,
 'NUCLEAR': 0,
 'STRIKE': 0,
 'FORCE': 0,
 'BUT': 0,
 'EUROPE': 0,
 'MADE': 0,
 'NO': 0,
 'ATTEMPT': 0,
 'DEVISE': 0,
 'A': 0,
 'PLAN': 0,
 'LAST': 0,
 'WEEK': 0,
 'AS': 0,
 'THEY': 0,
 'STUDIED': 0,
 'ACCORD': 0,
 'BETWEEN': 0,
 'PRESIDENT': 0,
 'KENNEDY': 0,
 'AND': 0,
 'PRIME': 0,
 'MINISTER': 0,
 'MACMILLAN': 0,
 'EUROPEANS': 0,
 'SAW': 0,
 'EMERGING': 0,
 'OUTLINES': 0,
 'OF': 0,
 'THAT': 0,
 'WANTS': 0,
 'WILL': 0,
 'SUPPORT': 0,
 'IT': 0,
 'ALL': 0,
 'SPRANG': 0,
 'FROM': 0,
 'ANGLOUS': 0,
 'CRISIS': 0,
 'OVER': 0,
 'CANCELLATION': 0,
 'BUGRIDDEN': 0,
 'SKYBOLT': 0,
 'MISSILE': 0,
 'OFFER': 0,
 'SUPPLY': 0,
 'BRITAIN': 0,
 'FRANCE': 0,
 'WITH': 0,
 'PROVED': 0,
 'POLARIS': 0,
 'TIME': 0,
 'DEC': 0,
 'ONE': 0,
 'ALLIED': 0,
 'LEADER': 0,
 'WHO': 0,
 'UNRESERVEDLY': 0,
 'WELCOMED': 0,
 'WAS': 0,


In [13]:
tfidf[0]

{'THE': 0.0,
 'ALLIES': 3.2188758248682006,
 'AFTER': 1.5324768712979722,
 'NASSAU': 6.437751649736401,
 'IN': 0.0,
 'DECEMBER': 1.6094379124341003,
 'US': 3.3471532697131465,
 'FIRST': 0.8925742052568391,
 'PROPOSED': 1.6094379124341003,
 'TO': 0.0,
 'HELP': 4.828313737302301,
 'NATO': 9.656627474604601,
 'DEVELOP': 3.2188758248682006,
 'ITS': 3.5757793663619353,
 'OWN': 3.6651629274966204,
 'NUCLEAR': 16.094379124341003,
 'STRIKE': 1.8325814637483102,
 'FORCE': 8.246616586867397,
 'BUT': 1.5324768712979722,
 'EUROPE': 4.828313737302301,
 'MADE': 1.6094379124341003,
 'NO': 2.5541281188299534,
 'ATTEMPT': 0.9162907318741551,
 'DEVISE': 1.6094379124341003,
 'A': 0.0,
 'PLAN': 0.9162907318741551,
 'LAST': 0.8925742052568391,
 'WEEK': 2.7488721956224653,
 'AS': 0.8925742052568391,
 'THEY': 0.6694306539426294,
 'STUDIED': 1.6094379124341003,
 'ACCORD': 1.6094379124341003,
 'BETWEEN': 0.9162907318741551,
 'PRESIDENT': 3.2188758248682006,
 'KENNEDY': 3.6651629274966204,
 'AND': 0.0,
 'PRIME'

In [ ]:
def pseudo_relevance_feedback(query, tfidf, vocab, k=10):
    # rel_docs, nrel_docs, query, tfidf, vocab, alpha=1, beta=0.75, gamma=0.15
    
    rel_docs = list(vector_space_model(query, k).keys())
    q_opt = relevance_feedback_rocchio(rel_docs=rel_docs, nrel_docs=[], query=query, tfidf=tfidf, vocab=vocab, gamma=0)
    return q_opt

## Stemming

In [ ]:
import string
import nltk
from nltk.tokenize import TreebankWordTokenizer

REMOVE_PUNCTUATION_TABLE = str.maketrans({x: None for x in string.punctuation})
TOKENIZER = TreebankWordTokenizer()

example_doc = docs[1]
example_doc_tokenized = TOKENIZER.tokenize(
        example_doc.translate(REMOVE_PUNCTUATION_TABLE))

In [ ]:
from nltk.stem.porter import PorterStemmer

STEMMER = PorterStemmer()
example_doc_tokenized_and_stemmed = [STEMMER.stem(token) for token
                                     in example_doc_tokenized]

In [ ]:
# tokenize and stem the query
def tokenize_and_stem(s):
    return [STEMMER.stem(t) for t 
            in TOKENIZER.tokenize(s.translate(REMOVE_PUNCTUATION_TABLE))]

tokenize_and_stem(query)


In [16]:
def wordList_removePuncs(doc_dict):
    stop = stopwords.words('english') + list(string.punctuation) + ['\n']
    wordList = []
    for doc in doc_dict.values():
        for word in word_tokenize(doc.lower().strip()): 
            if not word in stop:
                wordList.append(word)
    return wordList

# Evaluation

In [ ]:
def calculate_precision(k):
    """To generate list of precision values for each query for given value of k
    
    Arguments:
        k {[type]} -- number of top documents to be retrieved
    
    Returns:
        list -- list of precision values for each query
    """
    precision = []
    for i in range(len(queries)):
        
        # Number of relevant documents retrieved
        a = intersection(list_of_docs(k)[i][1].tolist(), query_rel[i])
        
        # Total number of documents retrieved
        b = k
        p = a / b
        precision.append(p)
    return precision

# for top 100 docs
calculate_precision(no_of_top)

In [ ]:
def calculate_recall(k):
    """To generate list of recall values for each query for given value of k
    
    Arguments:
        k {integer} -- number of top documents to be retrieved 
    
    Returns:
        list -- list of recall values for each query
    """
    
    recall = []
    for i in range(len(queries)):
        
        # Number of relevant documents retrieved
        a = intersection(list_of_docs(k)[i][1].tolist(), query_rel[i])
        
        # Total number of relevant documents
        b = len(query_rel[i])
        r = a / b
        recall.append(r)
    return recall   
# for top 100 docs
calculate_recall(no_of_top)

In [ ]:
def intersection(lst1, lst2): 
    """To count number of common items between 2 lists
    
    Arguments:
        lst1 {list} -- list 1
        lst2 {list} -- list 2
    
    Returns:
        integer -- number of common items between list 1 & list 2 
    """
    lst3 = [value for value in lst1 if value in lst2] 
    return len(lst3) 